# **DontREADME**

# Architecture du Projet ChatBot Documentaire

## 🏗️ Structure des dossiers

```
document-chatbot/
├── 📁 app/
│   ├── __init__.py
│   ├── main.py                    # Interface Gradio principale
│   ├── config.py                  # Configuration et constantes
│   └── components/
│       ├── __init__.py
│       ├── file_processor.py      # Traitement des fichiers (PDF, DOCX, TXT)
│       ├── vectorstore.py         # Gestion ChromaDB et embeddings
│       ├── chat_engine.py         # Logique de conversation et RAG
│       └── memory.py              # Gestion de l'historique
├── 📁 data/
│   ├── uploads/                   # Fichiers uploadés temporaires
│   └── vectorstore/               # Base ChromaDB persistante
├── 📁 utils/
│   ├── __init__.py
│   ├── text_splitter.py          # Découpage intelligent du texte
│   └── prompt_templates.py       # Templates de prompts pour le LLM
├── requirements.txt
├── .env.example
├── .gitignore
└── README.md
```

## 🔧 Architecture technique

### 1. Interface Utilisateur (Gradio)
```python
# Structure de l'interface
Interface Gradio:
├── Onglet "Configuration"
│   ├── Input: Clé API OpenAI/Mistral
│   ├── File Upload: PDF/DOCX/TXT
│   ├── Slider: Chunk Size (100-2000)
│   ├── Slider: Nombre de documents K (1-10)
│   └── Button: "Traiter le document"
├── Onglet "Chat"
│   ├── Chatbot: Historique des conversations
│   ├── Textbox: Question utilisateur
│   └── Button: "Envoyer"
└── Onglet "Informations"
    ├── Display: Statut du traitement
    ├── Display: Nombre de chunks créés
    └── Display: Modèle utilisé
```

### 2. Pipeline de traitement
```mermaid
graph TD
A[Upload Fichier] --> B[Extraction Texte]
B --> C[Découpage en Chunks]
C --> D[Génération Embeddings]
D --> E[Stockage ChromaDB]
E --> F[Prêt pour Questions]

G[Question Utilisateur] --> H[Recherche Similarité]
H --> I[Récupération Contexte]
I --> J[Génération Réponse LLM]
J --> K[Mise à jour Historique]
```

## 🧩 Composants principaux

### 1. **FileProcessor** (`file_processor.py`)
- **Responsabilité** : Extraction de texte des fichiers
- **Formats supportés** : PDF (PyPDF2), DOCX (python-docx), TXT
- **Fonctions** :
  - `extract_text_from_pdf()`
  - `extract_text_from_docx()`
  - `extract_text_from_txt()`
  - `process_uploaded_file()`

### 2. **VectorStore** (`vectorstore.py`)
- **Responsabilité** : Gestion des embeddings et ChromaDB
- **Fonctions** :
  - `initialize_vectorstore()`
  - `add_documents_to_vectorstore()`
  - `search_similar_documents()`
  - `clear_vectorstore()`

### 3. **ChatEngine** (`chat_engine.py`)
- **Responsabilité** : Logique RAG et interaction LLM
- **Fonctions** :
  - `setup_retrieval_chain()`
  - `process_question()`
  - `generate_response()`
  - `format_context()`

### 4. **Memory** (`memory.py`)
- **Responsabilité** : Historique des conversations
- **Fonctions** :
  - `add_message()`
  - `get_conversation_history()`
  - `clear_history()`
  - `format_history_for_display()`

## 🔄 Flow de données

### Étape 1: Initialisation
1. L'utilisateur saisit sa clé API
2. Configure les paramètres (chunk_size, k)
3. Upload un document

### Étape 2: Traitement du document
1. **FileProcessor** extrait le texte
2. **TextSplitter** découpe en chunks
3. **VectorStore** génère les embeddings via OpenAI/Mistral
4. Stockage dans ChromaDB

### Étape 3: Conversation
1. L'utilisateur pose une question
2. **VectorStore** recherche les chunks pertinents
3. **ChatEngine** combine contexte + historique + question
4. LLM génère la réponse
5. **Memory** sauvegarde l'échange

## 🛠️ Stack technique

- **Interface** : Gradio (plus simple que Streamlit pour ce cas)
- **LLM Framework** : LangChain
- **Vector Database** : ChromaDB (local, pas besoin de serveur)
- **LLM** : OpenAI GPT-3.5-turbo (ou Mistral AI en alternative gratuite)
- **Embeddings** : OpenAI text-embedding-ada-002 (ou sentence-transformers gratuit)
- **Document Processing** : PyPDF2, python-docx
- **Text Splitting** : LangChain RecursiveCharacterTextSplitter

## 📦 Dépendances principales

```txt
gradio>=4.0.0
langchain>=0.1.0
langchain-openai
langchain-community
chromadb>=0.4.0
openai>=1.0.0
PyPDF2>=3.0.0
python-docx>=0.8.11
sentence-transformers  # Alternative gratuite aux embeddings OpenAI
python-dotenv
```

## 🎯 Fonctionnalités avancées (optionnelles)

- **Multi-documents** : Traiter plusieurs fichiers simultanément
- **Export conversation** : Sauvegarder l'historique en JSON/PDF
- **Métriques** : Temps de réponse, similarité des chunks
- **Streaming** : Réponses en temps réel
- **Templates personnalisés** : Différents styles de réponse

Cette architecture modulaire permet une maintenance facile et des extensions futures. Voulez-vous que je commence par implémenter un composant spécifique ou préférez-vous que je crée l'application complète ?

## Scripts :

In [ ]:
# requirements.txt
"""
gradio>=4.0.0
langchain>=0.1.0
langchain-community
langchain-mistralai
chromadb>=0.4.0
mistralai>=0.1.0
PyPDF2>=3.0.0
python-docx>=0.8.11
sentence-transformers>=2.2.0
python-dotenv>=1.0.0
tiktoken>=0.5.0
"""

# app/config.py
import os
from dotenv import load_dotenv

load_dotenv()

class Config:
    # Mistral AI Configuration
    MISTRAL_API_KEY = os.getenv("MISTRAL_API_KEY", "")
    MISTRAL_MODEL = "mistral-tiny"  # ou "mistral-small", "mistral-medium"
    
    # ChromaDB Configuration
    CHROMADB_PATH = "./data/vectorstore"
    COLLECTION_NAME = "document_embeddings"
    
    # Text Processing
    DEFAULT_CHUNK_SIZE = 1000
    DEFAULT_CHUNK_OVERLAP = 200
    DEFAULT_K_DOCUMENTS = 3
    
    # Supported file types
    SUPPORTED_EXTENSIONS = ['.pdf', '.docx', '.txt']
    MAX_FILE_SIZE = 10 * 1024 * 1024  # 10MB

# app/components/file_processor.py
import os
import tempfile
from pathlib import Path
from typing import Optional, Tuple

import PyPDF2
from docx import Document

class FileProcessor:
    """Traitement des fichiers uploadés"""
    
    @staticmethod
    def extract_text_from_pdf(file_path: str) -> str:
        """Extrait le texte d'un fichier PDF"""
        try:
            with open(file_path, 'rb') as file:
                pdf_reader = PyPDF2.PdfReader(file)
                text = ""
                for page in pdf_reader.pages:
                    text += page.extract_text() + "\n"
                return text.strip()
        except Exception as e:
            raise Exception(f"Erreur lors de la lecture du PDF: {str(e)}")
    
    @staticmethod
    def extract_text_from_docx(file_path: str) -> str:
        """Extrait le texte d'un fichier DOCX"""
        try:
            doc = Document(file_path)
            text = ""
            for paragraph in doc.paragraphs:
                text += paragraph.text + "\n"
            return text.strip()
        except Exception as e:
            raise Exception(f"Erreur lors de la lecture du DOCX: {str(e)}")
    
    @staticmethod
    def extract_text_from_txt(file_path: str) -> str:
        """Extrait le texte d'un fichier TXT"""
        try:
            with open(file_path, 'r', encoding='utf-8') as file:
                return file.read().strip()
        except UnicodeDecodeError:
            # Essayer avec d'autres encodages
            for encoding in ['latin-1', 'cp1252', 'iso-8859-1']:
                try:
                    with open(file_path, 'r', encoding=encoding) as file:
                        return file.read().strip()
                except UnicodeDecodeError:
                    continue
            raise Exception("Impossible de décoder le fichier texte")
        except Exception as e:
            raise Exception(f"Erreur lors de la lecture du TXT: {str(e)}")
    
    @classmethod
    def process_uploaded_file(cls, file_obj) -> Tuple[str, str]:
        """
        Traite un fichier uploadé et retourne le texte extrait
        Returns: (text_content, filename)
        """
        if file_obj is None:
            raise ValueError("Aucun fichier fourni")
        
        # Sauvegarder temporairement le fichier
        with tempfile.NamedTemporaryFile(delete=False, suffix=Path(file_obj.name).suffix) as tmp_file:
            tmp_file.write(file_obj.read() if hasattr(file_obj, 'read') else file_obj)
            tmp_path = tmp_file.name
        
        try:
            file_extension = Path(file_obj.name).suffix.lower()
            filename = Path(file_obj.name).name
            
            if file_extension == '.pdf':
                text = cls.extract_text_from_pdf(tmp_path)
            elif file_extension == '.docx':
                text = cls.extract_text_from_docx(tmp_path)
            elif file_extension == '.txt':
                text = cls.extract_text_from_txt(tmp_path)
            else:
                raise ValueError(f"Format de fichier non supporté: {file_extension}")
            
            if not text.strip():
                raise ValueError("Le fichier ne contient pas de texte extractible")
            
            return text, filename
            
        finally:
            # Nettoyer le fichier temporaire
            if os.path.exists(tmp_path):
                os.unlink(tmp_path)

# app/components/vectorstore.py
import os
from typing import List, Optional
import chromadb
from chromadb.config import Settings
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.embeddings import HuggingFaceEmbeddings
from langchain_community.vectorstores import Chroma
from langchain.schema import Document

from app.config import Config

class VectorStoreManager:
    """Gestion de la base de données vectorielle ChromaDB"""
    
    def __init__(self):
        self.embeddings = None
        self.vectorstore = None
        self.text_splitter = None
        self._setup_embeddings()
    
    def _setup_embeddings(self):
        """Configure les embeddings avec sentence-transformers (gratuit)"""
        # Utilise un modèle multilingue français/anglais
        model_name = "sentence-transformers/paraphrase-multilingual-MiniLM-L12-v2"
        self.embeddings = HuggingFaceEmbeddings(
            model_name=model_name,
            model_kwargs={'device': 'cpu'},  # Utilise CPU pour la compatibilité
            encode_kwargs={'normalize_embeddings': True}
        )
    
    def setup_text_splitter(self, chunk_size: int = Config.DEFAULT_CHUNK_SIZE, 
                           chunk_overlap: int = Config.DEFAULT_CHUNK_OVERLAP):
        """Configure le découpeur de texte"""
        self.text_splitter = RecursiveCharacterTextSplitter(
            chunk_size=chunk_size,
            chunk_overlap=chunk_overlap,
            length_function=len,
            separators=["\n\n", "\n", ". ", " ", ""]
        )
    
    def initialize_vectorstore(self, collection_name: str = Config.COLLECTION_NAME):
        """Initialise la base vectorielle ChromaDB"""
        # Créer le dossier s'il n'existe pas
        os.makedirs(Config.CHROMADB_PATH, exist_ok=True)
        
        # Configuration ChromaDB
        client = chromadb.PersistentClient(path=Config.CHROMADB_PATH)
        
        # Supprimer la collection existante si elle existe
        try:
            client.delete_collection(collection_name)
        except ValueError:
            pass  # Collection n'existe pas
        
        self.vectorstore = Chroma(
            client=client,
            collection_name=collection_name,
            embedding_function=self.embeddings,
        )
    
    def add_documents(self, text: str, filename: str, chunk_size: int, chunk_overlap: int) -> int:
        """
        Ajoute des documents à la base vectorielle
        Returns: nombre de chunks créés
        """
        if not self.vectorstore:
            raise ValueError("VectorStore non initialisé")
        
        # Configuration du text splitter
        self.setup_text_splitter(chunk_size, chunk_overlap)
        
        # Découpage du texte
        chunks = self.text_splitter.split_text(text)
        
        # Création des documents avec métadonnées
        documents = [
            Document(
                page_content=chunk,
                metadata={
                    "filename": filename,
                    "chunk_id": i,
                    "total_chunks": len(chunks)
                }
            )
            for i, chunk in enumerate(chunks)
        ]
        
        # Ajout à la base vectorielle
        self.vectorstore.add_documents(documents)
        
        return len(chunks)
    
    def search_similar_documents(self, query: str, k: int = Config.DEFAULT_K_DOCUMENTS) -> List[Document]:
        """Recherche les documents similaires à la requête"""
        if not self.vectorstore:
            return []
        
        return self.vectorstore.similarity_search(query, k=k)
    
    def get_retriever(self, k: int = Config.DEFAULT_K_DOCUMENTS):
        """Retourne un retriever pour LangChain"""
        if not self.vectorstore:
            raise ValueError("VectorStore non initialisé")
        
        return self.vectorstore.as_retriever(search_kwargs={"k": k})

# app/components/memory.py
from typing import List, Dict, Any
from datetime import datetime

class ConversationMemory:
    """Gestion de l'historique des conversations"""
    
    def __init__(self):
        self.conversation_history: List[Dict[str, Any]] = []
    
    def add_exchange(self, question: str, answer: str, sources: List[str] = None):
        """Ajoute un échange question-réponse à l'historique"""
        exchange = {
            "timestamp": datetime.now().isoformat(),
            "question": question,
            "answer": answer,
            "sources": sources or []
        }
        self.conversation_history.append(exchange)
    
    def get_recent_history(self, n_exchanges: int = 3) -> str:
        """
        Retourne l'historique récent formaté pour le contexte LLM
        """
        if not self.conversation_history:
            return ""
        
        recent = self.conversation_history[-n_exchanges:]
        history_text = "Historique de la conversation:\n"
        
        for exchange in recent:
            history_text += f"Q: {exchange['question']}\n"
            history_text += f"R: {exchange['answer']}\n\n"
        
        return history_text
    
    def get_formatted_history(self) -> List[List[str]]:
        """Retourne l'historique formaté pour l'affichage Gradio"""
        formatted = []
        for exchange in self.conversation_history:
            formatted.append([exchange["question"], exchange["answer"]])
        return formatted
    
    def clear_history(self):
        """Efface l'historique"""
        self.conversation_history = []

# app/components/chat_engine.py
from typing import Optional, Tuple, List
from langchain_mistralai import ChatMistralAI
from langchain.chains import ConversationalRetrievalChain
from langchain.memory import ConversationBufferMemory
from langchain.prompts import PromptTemplate

from app.components.vectorstore import VectorStoreManager
from app.components.memory import ConversationMemory

class ChatEngine:
    """Moteur de conversation avec RAG"""
    
    def __init__(self):
        self.llm = None
        self.vectorstore_manager = VectorStoreManager()
        self.memory = ConversationMemory()
        self.chain = None
        
        # Template de prompt en français
        self.prompt_template = PromptTemplate(
            input_variables=["context", "question", "chat_history"],
            template="""Tu es un assistant IA spécialisé dans l'analyse de documents. 
Réponds aux questions en te basant UNIQUEMENT sur le contexte fourni.

Contexte des documents:
{context}

Historique de la conversation:
{chat_history}

Question: {question}

Instructions:
- Réponds de manière précise et concise
- Si l'information n'est pas dans le contexte, dis clairement "Je ne trouve pas cette information dans le document fourni"
- Cite des passages spécifiques quand c'est pertinent
- Reste factuel et objectif

Réponse:"""
        )
    
    def setup_llm(self, api_key: str, model: str = "mistral-tiny"):
        """Configure le modèle Mistral AI"""
        if not api_key:
            raise ValueError("Clé API Mistral requise")
        
        self.llm = ChatMistralAI(
            mistral_api_key=api_key,
            model=model,
            temperature=0.1,  # Réponses plus déterministes
            max_tokens=1000
        )
    
    def setup_chain(self, k_documents: int = 3):
        """Configure la chaîne de traitement RAG"""
        if not self.llm:
            raise ValueError("LLM non configuré")
        
        if not self.vectorstore_manager.vectorstore:
            raise ValueError("VectorStore non initialisé")
        
        # Mémoire pour la conversation
        memory = ConversationBufferMemory(
            memory_key="chat_history",
            return_messages=True,
            output_key="answer"
        )
        
        # Chaîne conversationnelle avec retrieval
        self.chain = ConversationalRetrievalChain.from_llm(
            llm=self.llm,
            retriever=self.vectorstore_manager.get_retriever(k=k_documents),
            memory=memory,
            return_source_documents=True,
            combine_docs_chain_kwargs={"prompt": self.prompt_template}
        )
    
    def process_question(self, question: str) -> Tuple[str, List[str]]:
        """
        Traite une question et retourne la réponse avec les sources
        Returns: (answer, sources)
        """
        if not self.chain:
            return "Erreur: Système non configuré", []
        
        if not question.strip():
            return "Veuillez poser une question.", []
        
        try:
            # Exécution de la chaîne
            result = self.chain({"question": question})
            
            answer = result.get("answer", "Pas de réponse générée")
            
            # Extraction des sources
            sources = []
            if "source_documents" in result:
                for doc in result["source_documents"]:
                    filename = doc.metadata.get("filename", "Document")
                    chunk_id = doc.metadata.get("chunk_id", 0)
                    sources.append(f"{filename} (section {chunk_id + 1})")
            
            # Sauvegarde dans l'historique
            self.memory.add_exchange(question, answer, sources)
            
            return answer, sources
            
        except Exception as e:
            error_msg = f"Erreur lors du traitement: {str(e)}"
            return error_msg, []
    
    def get_conversation_history(self):
        """Retourne l'historique formaté pour Gradio"""
        return self.memory.get_formatted_history()
    
    def clear_conversation(self):
        """Efface l'historique de conversation"""
        self.memory.clear_history()
        if self.chain and hasattr(self.chain, 'memory'):
            self.chain.memory.clear()

# app/main.py
import gradio as gr
import os
from typing import Optional, Tuple, List

from app.components.file_processor import FileProcessor
from app.components.chat_engine import ChatEngine
from app.config import Config

class DocumentChatBot:
    """Application principale du ChatBot documentaire"""
    
    def __init__(self):
        self.chat_engine = ChatEngine()
        self.current_document = None
        self.document_processed = False
        
    def process_document(self, 
                        api_key: str, 
                        file_obj, 
                        chunk_size: int, 
                        k_documents: int) -> Tuple[str, str]:
        """
        Traite le document uploadé
        Returns: (status_message, document_info)
        """
        try:
            # Validation des inputs
            if not api_key.strip():
                return "❌ Veuillez saisir votre clé API Mistral", ""
            
            if file_obj is None:
                return "❌ Veuillez sélectionner un fichier", ""
            
            # Configuration du LLM
            self.chat_engine.setup_llm(api_key)
            
            # Traitement du fichier
            text_content, filename = FileProcessor.process_uploaded_file(file_obj)
            
            # Initialisation de la base vectorielle
            self.chat_engine.vectorstore_manager.initialize_vectorstore()
            
            # Ajout des documents
            chunk_overlap = max(50, chunk_size // 5)  # 20% de chevauchement
            num_chunks = self.chat_engine.vectorstore_manager.add_documents(
                text_content, filename, chunk_size, chunk_overlap
            )
            
            # Configuration de la chaîne de traitement
            self.chat_engine.setup_chain(k_documents)
            
            # État mis à jour
            self.current_document = filename
            self.document_processed = True
            
            status = f"✅ Document traité avec succès!"
            doc_info = f"""
📄 **Fichier**: {filename}
📊 **Chunks créés**: {num_chunks}
⚙️ **Taille des chunks**: {chunk_size} caractères
🔍 **Documents récupérés**: {k_documents}
🤖 **Modèle**: Mistral AI (mistral-tiny)
            """.strip()
            
            return status, doc_info
            
        except Exception as e:
            error_msg = f"❌ Erreur: {str(e)}"
            return error_msg, ""
    
    def chat_with_document(self, message: str, history: List[List[str]]) -> Tuple[str, List[List[str]]]:
        """
        Traite une question du chat
        Returns: ("", updated_history)
        """
        if not self.document_processed:
            history.append([message, "⚠️ Veuillez d'abord traiter un document dans l'onglet Configuration."])
            return "", history
        
        if not message.strip():
            return "", history
        
        # Traitement de la question
        answer, sources = self.chat_engine.process_question(message)
        
        # Formatage de la réponse avec sources
        if sources:
            formatted_answer = f"{answer}\n\n📚 **Sources**: {', '.join(sources)}"
        else:
            formatted_answer = answer
        
        # Mise à jour de l'historique
        history.append([message, formatted_answer])
        
        return "", history
    
    def clear_chat(self) -> List[List[str]]:
        """Efface l'historique du chat"""
        if self.chat_engine:
            self.chat_engine.clear_conversation()
        return []
    
    def create_interface(self):
        """Crée l'interface Gradio"""
        
        # CSS personnalisé
        css = """
        .gradio-container {
            font-family: 'Segoe UI', Tahoma, Geneva, Verdana, sans-serif;
        }
        .tab-nav button {
            font-size: 16px;
            font-weight: 500;
        }
        """
        
        with gr.Blocks(css=css, title="ChatBot Documentaire - Mistral AI", theme=gr.themes.Soft()) as interface:
            
            gr.Markdown("""
            # 📄 ChatBot Documentaire avec Mistral AI
            
            Posez des questions sur vos documents PDF, DOCX et TXT grâce à l'IA !
            """)
            
            with gr.Tabs():
                
                # Onglet Configuration
                with gr.Tab("⚙️ Configuration", id="config"):
                    
                    with gr.Row():
                        with gr.Column(scale=2):
                            api_key_input = gr.Textbox(
                                label="🔑 Clé API Mistral AI",
                                placeholder="Saisissez votre clé API Mistral...",
                                type="password",
                                info="Obtenez votre clé gratuite sur https://console.mistral.ai/"
                            )
                            
                            file_upload = gr.File(
                                label="📁 Fichier à analyser",
                                file_types=[".pdf", ".docx", ".txt"],
                                type="binary"
                            )
                            
                            with gr.Row():
                                chunk_size_slider = gr.Slider(
                                    minimum=200,
                                    maximum=2000,
                                    value=Config.DEFAULT_CHUNK_SIZE,
                                    step=100,
                                    label="📏 Taille des chunks",
                                    info="Taille des segments de texte"
                                )
                                
                                k_documents_slider = gr.Slider(
                                    minimum=1,
                                    maximum=10,
                                    value=Config.DEFAULT_K_DOCUMENTS,
                                    step=1,
                                    label="🔍 Nombre de documents récupérés",
                                    info="Nombre de segments utilisés pour répondre"
                                )
                            
                            process_btn = gr.Button("🚀 Traiter le document", variant="primary", size="lg")
                        
                        with gr.Column(scale=1):
                            status_output = gr.Textbox(
                                label="📊 Statut",
                                interactive=False,
                                lines=2
                            )
                            
                            doc_info_output = gr.Markdown(
                                label="ℹ️ Informations du document"
                            )
                
                # Onglet Chat
                with gr.Tab("💬 Chat", id="chat"):
                    
                    chatbot = gr.Chatbot(
                        label="Conversation avec votre document",
                        height=500,
                        show_copy_button=True
                    )
                    
                    with gr.Row():
                        msg_input = gr.Textbox(
                            label="Votre question",
                            placeholder="Posez une question sur votre document...",
                            scale=4
                        )
                        
                        with gr.Column(scale=1):
                            send_btn = gr.Button("📤 Envoyer", variant="primary")
                            clear_btn = gr.Button("🗑️ Effacer", variant="secondary")
                
                # Onglet Aide
                with gr.Tab("❓ Aide", id="help"):
                    gr.Markdown("""
                    ## 🚀 Comment utiliser cette application ?
                    
                    ### 1. Configuration
                    - **Obtenez une clé API Mistral AI** (gratuite) sur [console.mistral.ai](https://console.mistral.ai/)
                    - **Uploadez votre document** (PDF, DOCX ou TXT, max 10MB)
                    - **Ajustez les paramètres** selon vos besoins
                    - **Cliquez sur "Traiter le document"**
                    
                    ### 2. Chat
                    - **Posez vos questions** en langage naturel
                    - L'IA répondra en se basant **uniquement** sur votre document
                    - Les **sources** sont indiquées pour chaque réponse
                    
                    ### 3. Conseils
                    - **Questions précises** = meilleures réponses
                    - **Reformulez** si la réponse ne vous convient pas
                    - **Historique** conservé pendant la session
                    
                    ### 🔧 Paramètres avancés
                    - **Taille des chunks**: Plus petit = plus précis, plus grand = plus de contexte
                    - **Nombre de documents**: Plus = plus de contexte, mais peut diluer la réponse
                    
                    ### 🆓 Modèle utilisé
                    **Mistral AI (mistral-tiny)** - Modèle français performant et gratuit !
                    """)
            
            # Événements
            process_btn.click(
                fn=self.process_document,
                inputs=[api_key_input, file_upload, chunk_size_slider, k_documents_slider],
                outputs=[status_output, doc_info_output]
            )
            
            send_btn.click(
                fn=self.chat_with_document,
                inputs=[msg_input, chatbot],
                outputs=[msg_input, chatbot]
            )
            
            msg_input.submit(
                fn=self.chat_with_document,
                inputs=[msg_input, chatbot],
                outputs=[msg_input, chatbot]
            )
            
            clear_btn.click(
                fn=self.clear_chat,
                outputs=[chatbot]
            )
        
        return interface

def main():
    """Point d'entrée principal"""
    # Créer les dossiers nécessaires
    os.makedirs("./data/uploads", exist_ok=True)
    os.makedirs("./data/vectorstore", exist_ok=True)
    
    # Lancer l'application
    app = DocumentChatBot()
    interface = app.create_interface()
    
    interface.launch(
        server_name="0.0.0.0",  # Accessible depuis le réseau
        server_port=7860,
        share=False,  # Mettre True pour un lien public temporaire
        debug=True
    )

if __name__ == "__main__":
    main()

## Partie utils :

In [ ]:
# utils/__init__.py
"""
Package utilitaires pour le ChatBot documentaire
"""

from .text_splitter import SmartTextSplitter
from .prompt_templates import PromptTemplateManager
from .validators import FileValidator, InputValidator
from .performance import PerformanceMonitor

__all__ = [
    'SmartTextSplitter',
    'PromptTemplateManager', 
    'FileValidator',
    'InputValidator',
    'PerformanceMonitor'
]

# utils/text_splitter.py
import re
from typing import List, Dict, Any, Optional
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.schema import Document

class SmartTextSplitter:
    """
    Découpeur de texte intelligent avec optimisations pour différents types de documents
    """
    
    def __init__(self):
        self.separators = {
            'default': ["\n\n", "\n", ". ", "! ", "? ", " ", ""],
            'academic': ["\n\n", "\n", ". ", "; ", ", ", " ", ""],
            'technical': ["\n\n", "\n", ".\n", ". ", ":\n", ": ", " ", ""],
            'legal': ["\n\n", "\n", ". ", "; ", " - ", " ", ""]
        }
    
    def detect_document_type(self, text: str) -> str:
        """
        Détecte le type de document pour optimiser le découpage
        """
        text_lower = text.lower()
        
        # Mots-clés pour différents types
        academic_keywords = ['abstract', 'résumé', 'introduction', 'conclusion', 'références', 'bibliographie']
        technical_keywords = ['api', 'fonction', 'class', 'def ', 'import', 'documentation', 'manuel']
        legal_keywords = ['article', 'clause', 'alinéa', 'considérant', 'attendu', 'arrêté']
        
        # Compter les occurrences
        academic_score = sum(1 for keyword in academic_keywords if keyword in text_lower)
        technical_score = sum(1 for keyword in technical_keywords if keyword in text_lower)
        legal_score = sum(1 for keyword in legal_keywords if keyword in text_lower)
        
        # Déterminer le type
        scores = {
            'academic': academic_score,
            'technical': technical_score,
            'legal': legal_score
        }
        
        max_score = max(scores.values())
        if max_score >= 2:  # Seuil minimum
            return max(scores, key=scores.get)
        
        return 'default'
    
    def create_splitter(self, 
                       chunk_size: int = 1000,
                       chunk_overlap: int = 200,
                       document_type: str = 'auto') -> RecursiveCharacterTextSplitter:
        """
        Crée un text splitter optimisé selon le type de document
        """
        if document_type == 'auto':
            # Le type sera détecté lors du split_text
            separators = self.separators['default']
        else:
            separators = self.separators.get(document_type, self.separators['default'])
        
        return RecursiveCharacterTextSplitter(
            chunk_size=chunk_size,
            chunk_overlap=chunk_overlap,
            length_function=len,
            separators=separators,
            keep_separator=True
        )
    
    def smart_split(self, 
                   text: str, 
                   chunk_size: int = 1000,
                   chunk_overlap: int = 200,
                   preserve_structure: bool = True) -> List[str]:
        """
        Découpage intelligent qui préserve la structure du document
        """
        # Détection automatique du type
        doc_type = self.detect_document_type(text)
        
        # Préprocessing pour préserver la structure
        if preserve_structure:
            text = self._preprocess_text(text)
        
        # Création du splitter adapté
        splitter = self.create_splitter(chunk_size, chunk_overlap, doc_type)
        
        # Découpage
        chunks = splitter.split_text(text)
        
        # Post-processing pour nettoyer les chunks
        cleaned_chunks = [self._clean_chunk(chunk) for chunk in chunks]
        
        return [chunk for chunk in cleaned_chunks if chunk.strip()]
    
    def split_documents_with_metadata(self,
                                    text: str,
                                    filename: str,
                                    chunk_size: int = 1000,
                                    chunk_overlap: int = 200) -> List[Document]:
        """
        Découpe le texte et crée des documents avec métadonnées enrichies
        """
        chunks = self.smart_split(text, chunk_size, chunk_overlap)
        doc_type = self.detect_document_type(text)
        
        documents = []
        for i, chunk in enumerate(chunks):
            # Métadonnées enrichies
            metadata = {
                "filename": filename,
                "chunk_id": i,
                "total_chunks": len(chunks),
                "document_type": doc_type,
                "chunk_size": len(chunk),
                "chunk_position": "start" if i == 0 else ("end" if i == len(chunks) - 1 else "middle"),
                "contains_structure": self._has_structure_markers(chunk)
            }
            
            # Ajout de mots-clés pour ce chunk
            keywords = self._extract_keywords(chunk)
            if keywords:
                metadata["keywords"] = keywords
            
            documents.append(Document(page_content=chunk, metadata=metadata))
        
        return documents
    
    def _preprocess_text(self, text: str) -> str:
        """Préprocessing pour améliorer le découpage"""
        # Normaliser les espaces
        text = re.sub(r'\s+', ' ', text)
        
        # Préserver les sauts de ligne importants
        text = re.sub(r'\n\s*\n', '\n\n', text)
        
        # Améliorer la détection des phrases
        text = re.sub(r'([.!?])\s*([A-Z])', r'\1\n\2', text)
        
        return text.strip()
    
    def _clean_chunk(self, chunk: str) -> str:
        """Nettoie un chunk après découpage"""
        # Supprimer les espaces en début/fin
        chunk = chunk.strip()
        
        # Supprimer les lignes vides multiples
        chunk = re.sub(r'\n\s*\n\s*\n', '\n\n', chunk)
        
        # S'assurer qu'on ne commence pas par un séparateur
        chunk = re.sub(r'^[.!?;:,\s]+', '', chunk)
        
        return chunk
    
    def _has_structure_markers(self, chunk: str) -> bool:
        """Détecte si le chunk contient des marqueurs structurels"""
        structure_patterns = [
            r'^\d+\.',  # Numérotation
            r'^[A-Z][.]',  # Sections A., B., etc.
            r'^-\s',  # Listes à puces
            r'^\*\s',  # Listes étoiles
            r'^\w+:\s',  # Titre: contenu
        ]
        
        return any(re.search(pattern, chunk, re.MULTILINE) for pattern in structure_patterns)
    
    def _extract_keywords(self, chunk: str, max_keywords: int = 5) -> List[str]:
        """Extrait les mots-clés principaux d'un chunk"""
        # Mots vides français et anglais
        stop_words = {
            'le', 'la', 'les', 'un', 'une', 'des', 'de', 'du', 'et', 'ou', 'à', 'dans', 'sur', 'pour', 'par',
            'avec', 'sans', 'sous', 'que', 'qui', 'quoi', 'dont', 'où', 'ce', 'cette', 'ces', 'est', 'sont',
            'the', 'a', 'an', 'and', 'or', 'but', 'in', 'on', 'at', 'to', 'for', 'of', 'with', 'by', 'is', 'are'
        }
        
        # Extraction des mots
        words = re.findall(r'\b[a-zA-ZàâäéèêëïîôöùûüÿçÀÂÄÉÈÊËÏÎÔÖÙÛÜŸÇ]{3,}\b', chunk.lower())
        
        # Filtrage et comptage
        word_freq = {}
        for word in words:
            if word not in stop_words and len(word) > 2:
                word_freq[word] = word_freq.get(word, 0) + 1
        
        # Retourner les mots les plus fréquents
        sorted_words = sorted(word_freq.items(), key=lambda x: x[1], reverse=True)
        return [word for word, _ in sorted_words[:max_keywords]]

# utils/prompt_templates.py
from langchain.prompts import PromptTemplate
from typing import Dict, List

class PromptTemplateManager:
    """
    Gestionnaire de templates de prompts optimisés pour différents cas d'usage
    """
    
    def __init__(self):
        self.templates = self._initialize_templates()
    
    def _initialize_templates(self) -> Dict[str, PromptTemplate]:
        """Initialise les différents templates de prompts"""
        
        templates = {}
        
        # Template général (par défaut)
        templates['general'] = PromptTemplate(
            input_variables=["context", "question", "chat_history"],
            template="""Tu es un assistant IA spécialisé dans l'analyse de documents. 
Réponds aux questions en te basant UNIQUEMENT sur le contexte fourni.

Contexte des documents:
{context}

Historique de la conversation:
{chat_history}

Question: {question}

Instructions:
- Réponds de manière précise et concise
- Si l'information n'est pas dans le contexte, dis clairement "Je ne trouve pas cette information dans le document fourni"
- Cite des passages spécifiques quand c'est pertinent
- Reste factuel et objectif

Réponse:"""
        )
        
        # Template pour documents académiques
        templates['academic'] = PromptTemplate(
            input_variables=["context", "question", "chat_history"],
            template="""Tu es un assistant de recherche académique spécialisé dans l'analyse de publications scientifiques.
Analyse le contexte fourni pour répondre à la question de manière rigoureuse.

Contexte du document académique:
{context}

Historique de la conversation:
{chat_history}

Question: {question}

Instructions:
- Fournis une réponse académique rigoureuse
- Cite les auteurs et sections spécifiques si mentionnés
- Distingue clairement les faits des hypothèses
- Si des données ou statistiques sont présentes, inclus-les dans ta réponse
- Indique les limites de l'information disponible
- Si l'information n'est pas dans le document, dis "Cette information n'est pas présente dans le document analysé"

Réponse académique:"""
        )
        
        # Template pour documents techniques
        templates['technical'] = PromptTemplate(
            input_variables=["context", "question", "chat_history"],
            template="""Tu es un assistant technique expert en documentation et manuels.
Analyse la documentation technique pour fournir une réponse précise et applicable.

Documentation technique:
{context}

Historique de la conversation:
{chat_history}

Question technique: {question}

Instructions:
- Fournis une réponse technique précise et actionnable
- Inclus les étapes, commandes ou configurations si pertinentes
- Mentionne les prérequis ou limitations
- Utilise la terminologie technique appropriée
- Si des exemples de code sont présents, inclus-les
- Si l'information technique n'est pas disponible, dis "Cette information technique n'est pas documentée dans ce manuel"

Réponse technique:"""
        )
        
        # Template pour documents légaux
        templates['legal'] = PromptTemplate(
            input_variables=["context", "question", "chat_history"],
            template="""Tu es un assistant spécialisé dans l'analyse de documents juridiques.
Analyse le texte légal pour répondre de manière précise et structurée.

Texte légal:
{context}

Historique de la conversation:
{chat_history}

Question juridique: {question}

Instructions:
- Fournis une réponse structurée et précise
- Cite les articles, clauses ou sections spécifiques
- Respecte la terminologie juridique
- Distingue les obligations, droits et procédures
- Ne fournis pas de conseil juridique, seulement l'analyse du texte
- Si l'information n'est pas dans le document, dis "Cette disposition n'est pas couverte dans ce texte"

Analyse juridique:"""
        )
        
        # Template pour résumé
        templates['summary'] = PromptTemplate(
            input_variables=["context"],
            template="""Tu dois créer un résumé structuré et complet du document fourni.

Contenu du document:
{context}

Instructions:
- Crée un résumé structuré avec des sections claires
- Inclus les points clés et informations importantes
- Utilise des puces pour les listes d'éléments
- Maintiens l'objectivité et la précision
- Indique s'il s'agit d'un résumé partiel si le document semble incomplet

Structure suggérée:
## Résumé du document

### Points principaux
- [Point 1]
- [Point 2]
- [etc.]

### Informations clés
- [Information 1]
- [Information 2]
- [etc.]

### Conclusion
[Synthèse globale]

Résumé:"""
        )
        
        # Template pour extraction d'informations spécifiques
        templates['extraction'] = PromptTemplate(
            input_variables=["context", "question"],
            template="""Tu dois extraire des informations spécifiques du document selon la demande.

Document:
{context}

Demande d'extraction: {question}

Instructions:
- Extrais UNIQUEMENT les informations demandées
- Présente les résultats de manière structurée
- Si l'information n'est pas présente, indique "Information non trouvée"
- Utilise des listes ou tableaux si approprié
- Reste fidèle au texte original sans interpréter

Extraction:"""
        )
        
        return templates
    
    def get_template(self, template_type: str = 'general') -> PromptTemplate:
        """
        Retourne le template demandé
        """
        return self.templates.get(template_type, self.templates['general'])
    
    def get_available_templates(self) -> List[str]:
        """
        Retourne la liste des templates disponibles
        """
        return list(self.templates.keys())
    
    def create_custom_template(self, 
                             template_name: str,
                             template_content: str,
                             input_variables: List[str]) -> None:
        """
        Ajoute un template personnalisé
        """
        self.templates[template_name] = PromptTemplate(
            input_variables=input_variables,
            template=template_content
        )
    
    def get_optimized_template(self, document_type: str, query_type: str = 'general') -> PromptTemplate:
        """
        Retourne le template le mieux adapté selon le type de document et de requête
        """
        # Logique de sélection du template optimal
        if query_type == 'summary':
            return self.get_template('summary')
        elif query_type == 'extraction':
            return self.get_template('extraction')
        elif document_type in ['academic', 'technical', 'legal']:
            return self.get_template(document_type)
        else:
            return self.get_template('general')

# utils/validators.py
import os
import magic
from pathlib import Path
from typing import Tuple, Optional, List
import re

class FileValidator:
    """
    Validateur pour les fichiers uploadés
    """
    
    SUPPORTED_EXTENSIONS = {'.pdf', '.docx', '.txt', '.doc'}
    MAX_FILE_SIZE = 10 * 1024 * 1024  # 10MB
    MIN_FILE_SIZE = 100  # 100 bytes
    
    @classmethod
    def validate_file(cls, file_obj) -> Tuple[bool, Optional[str]]:
        """
        Valide un fichier uploadé
        Returns: (is_valid, error_message)
        """
        if file_obj is None:
            return False, "Aucun fichier fourni"
        
        # Vérification de l'extension
        if hasattr(file_obj, 'name'):
            file_extension = Path(file_obj.name).suffix.lower()
            if file_extension not in cls.SUPPORTED_EXTENSIONS:
                return False, f"Format non supporté. Formats acceptés: {', '.join(cls.SUPPORTED_EXTENSIONS)}"
        
        # Vérification de la taille
        try:
            if hasattr(file_obj, 'size'):
                file_size = file_obj.size
            else:
                # Fallback pour les objets sans attribut size
                content = file_obj.read() if hasattr(file_obj, 'read') else file_obj
                file_size = len(content)
                # Reset du pointeur si possible
                if hasattr(file_obj, 'seek'):
                    file_obj.seek(0)
            
            if file_size > cls.MAX_FILE_SIZE:
                return False, f"Fichier trop volumineux (max {cls.MAX_FILE_SIZE // (1024*1024)}MB)"
            
            if file_size < cls.MIN_FILE_SIZE:
                return False, "Fichier trop petit ou vide"
                
        except Exception as e:
            return False, f"Erreur lors de la vérification de la taille: {str(e)}"
        
        return True, None
    
    @classmethod
    def detect_file_type(cls, file_path: str) -> str:
        """
        Détecte le type MIME d'un fichier
        """
        try:
            mime_type = magic.from_file(file_path, mime=True)
            return mime_type
        except:
            # Fallback basé sur l'extension
            extension = Path(file_path).suffix.lower()
            mime_mapping = {
                '.pdf': 'application/pdf',
                '.docx': 'application/vnd.openxmlformats-officedocument.wordprocessingml.document',
                '.doc': 'application/msword',
                '.txt': 'text/plain'
            }
            return mime_mapping.get(extension, 'unknown')

class InputValidator:
    """
    Validateur pour les entrées utilisateur
    """
    
    @staticmethod
    def validate_api_key(api_key: str, provider: str = 'mistral') -> Tuple[bool, Optional[str]]:
        """
        Valide une clé API
        """
        if not api_key or not api_key.strip():
            return False, "Clé API manquante"
        
        api_key = api_key.strip()
        
        if provider.lower() == 'mistral':
            # Format attendu pour Mistral: commence généralement par des caractères spécifiques
            if len(api_key) < 20:
                return False, "Clé API Mistral trop courte"
            
            # Vérification basique du format
            if not re.match(r'^[a-zA-Z0-9_-]+$', api_key):
                return False, "Format de clé API invalide"
        
        elif provider.lower() == 'openai':
            # Format OpenAI: sk-...
            if not api_key.startswith('sk-'):
                return False, "Clé API OpenAI doit commencer par 'sk-'"
            
            if len(api_key) < 45:
                return False, "Clé API OpenAI trop courte"
        
        return True, None
    
    @staticmethod
    def validate_chunk_size(chunk_size: int) -> Tuple[bool, Optional[str]]:
        """
        Valide la taille des chunks
        """
        if not isinstance(chunk_size, int):
            return False, "La taille des chunks doit être un nombre entier"
        
        if chunk_size < 100:
            return False, "Taille des chunks trop petite (minimum 100)"
        
        if chunk_size > 4000:
            return False, "Taille des chunks trop grande (maximum 4000)"
        
        return True, None
    
    @staticmethod
    def validate_k_documents(k: int) -> Tuple[bool, Optional[str]]:
        """
        Valide le nombre de documents à récupérer
        """
        if not isinstance(k, int):
            return False, "Le nombre de documents doit être un entier"
        
        if k < 1:
            return False, "Nombre de documents minimum: 1"
        
        if k > 20:
            return False, "Nombre de documents maximum: 20"
        
        return True, None
    
    @staticmethod
    def validate_question(question: str) -> Tuple[bool, Optional[str]]:
        """
        Valide une question utilisateur
        """
        if not question or not question.strip():
            return False, "Question vide"
        
        question = question.strip()
        
        if len(question) < 3:
            return False, "Question trop courte (minimum 3 caractères)"
        
        if len(question) > 1000:
            return False, "Question trop longue (maximum 1000 caractères)"
        
        # Vérification de caractères suspects
        suspicious_patterns = [
            r'<script',
            r'javascript:',
            r'eval\(',
            r'exec\('
        ]
        
        for pattern in suspicious_patterns:
            if re.search(pattern, question, re.IGNORECASE):
                return False, "Question contient des éléments suspects"
        
        return True, None
    
    @staticmethod
    def sanitize_input(text: str) -> str:
        """
        Nettoie et sécurise une entrée texte
        """
        if not isinstance(text, str):
            return ""
        
        # Supprime les caractères de contrôle
        text = re.sub(r'[\x00-\x08\x0B\x0C\x0E-\x1F\x7F]', '', text)
        
        # Normalise les espaces
        text = re.sub(r'\s+', ' ', text)
        
        # Supprime les espaces en début/fin
        text = text.strip()
        
        return text

# utils/performance.py
import time
import psutil
import threading
from typing import Dict, List, Optional, Callable
from dataclasses import dataclass
from datetime import datetime, timedelta

@dataclass
class PerformanceMetrics:
    """Structure pour stocker les métriques de performance"""
    timestamp: datetime
    operation: str
    duration: float
    memory_usage: float
    cpu_usage: float
    success: bool
    error_message: Optional[str] = None

class PerformanceMonitor:
    """
    Moniteur de performance pour l'application
    """
    
    def __init__(self, max_history: int = 100):
        self.metrics_history: List[PerformanceMetrics] = []
        self.max_history = max_history
        self._lock = threading.Lock()
    
    def measure_performance(self, operation_name: str):
        """
        Décorateur pour mesurer les performances d'une fonction
        """
        def decorator(func: Callable):
            def wrapper(*args, **kwargs):
                start_time = time.time()
                start_memory = psutil.Process().memory_info().rss / 1024 / 1024  # MB
                start_cpu = psutil.cpu_percent()
                
                success = True
                error_message = None
                result = None
                
                try:
                    result = func(*args, **kwargs)
                except Exception as e:
                    success = False
                    error_message = str(e)
                    raise
                finally:
                    end_time = time.time()
                    end_memory = psutil.Process().memory_info().rss / 1024 / 1024  # MB
                    duration = end_time - start_time
                    
                    # Mesure CPU pendant l'opération (approximation)
                    cpu_usage = psutil.cpu_percent()
                    
                    metrics = PerformanceMetrics(
                        timestamp=datetime.now(),
                        operation=operation_name,
                        duration=duration,
                        memory_usage=end_memory - start_memory,
                        cpu_usage=cpu_usage,
                        success=success,
                        error_message=error_message
                    )
                    
                    self._add_metrics(metrics)
                
                return result
            return wrapper
        return decorator
    
    def _add_metrics(self, metrics: PerformanceMetrics):
        """Ajoute des métriques à l'historique"""
        with self._lock:
            self.metrics_history.append(metrics)
            
            # Limiter la taille de l'historique
            if len(self.metrics_history) > self.max_history:
                self.metrics_history = self.metrics_history[-self.max_history:]
    
    def get_metrics_summary(self, 
                           operation: Optional[str] = None,
                           last_n_minutes: Optional[int] = None) -> Dict:
        """
        Retourne un résumé des métriques
        """
        with self._lock:
            filtered_metrics = self.metrics_history.copy()
        
        # Filtrage par opération
        if operation:
            filtered_metrics = [m for m in filtered_metrics if m.operation == operation]
        
        # Filtrage temporel
        if last_n_minutes:
            cutoff_time = datetime.now() - timedelta(minutes=last_n_minutes)
            filtered_metrics = [m for m in filtered_metrics if m.timestamp >= cutoff_time]
        
        if not filtered_metrics:
            return {"message": "Aucune métrique disponible"}
        
        # Calculs statistiques
        durations = [m.duration for m in filtered_metrics]
        memory_usages = [m.memory_usage for m in filtered_metrics]
        success_count = sum(1 for m in filtered_metrics if m.success)
        
        summary = {
            "total_operations": len(filtered_metrics),
            "success_rate": success_count / len(filtered_metrics) * 100,
            "avg_duration": sum(durations) / len(durations),
            "max_duration": max(durations),
            "min_duration": min(durations),
            "avg_memory_usage": sum(memory_usages) / len(memory_usages),
            "max_memory_usage": max(memory_usages),
            "recent_errors": [
                {"operation": m.operation, "error": m.error_message, "timestamp": m.timestamp}
                for m in filtered_metrics[-5:] if not m.success
            ]
        }
        
        return summary
    
    def get_system_info(self) -> Dict:
        """Retourne les informations système actuelles"""
        return {
            "cpu_percent": psutil.cpu_percent(),
            "memory_percent": psutil.virtual_memory().percent,
            "memory_available_gb": psutil.virtual_memory().available / 1024 / 1024 / 1024,
            "disk_usage_percent": psutil.disk_usage('/').percent,
            "timestamp": datetime.now().isoformat()
        }
    
    def export_metrics(self, filepath: str):
        """Exporte les métriques vers un fichier JSON"""
        import json
        
        with self._lock:
            data = {
                "export_timestamp": datetime.now().isoformat(),
                "metrics": [
                    {
                        "timestamp": m.timestamp.isoformat(),
                        "operation": m.operation,
                        "duration": m.duration,
                        "memory_usage": m.memory_usage,
                        "cpu_usage": m.cpu_usage,
                        "success": m.success,
                        "error_message": m.error_message
                    }
                    for m in self.metrics_history
                ]
            }
        
        with open(filepath, 'w', encoding='utf-8') as f:
            json.dump(data, f, indent=2, ensure_ascii=False)
    
    def clear_metrics(self):
        """Efface l'historique des métriques"""
        with self._lock:
            self.metrics_history.clear()

# Exemple d'utilisation du moniteur de performance
def create_performance_monitor():
    """Factory function pour créer un moniteur de performance"""
    return PerformanceMonitor(max_history=200)

📋 Récapitulatif des scripts créés :
1. text_splitter.py - Découpage intelligent

SmartTextSplitter : Détection automatique du type de document
Types supportés : académique, technique, légal, général
Optimisations : préservation de structure, extraction de mots-clés
Métadonnées enrichies pour chaque chunk

2. prompt_templates.py - Templates de prompts optimisés

6 templates spécialisés : général, académique, technique, légal, résumé, extraction
Sélection automatique selon le type de document
Templates personnalisables
Optimisation contextuelle

3. validators.py - Validation et sécurité

FileValidator : validation fichiers (taille, format, MIME)
InputValidator : validation des entrées (API keys, paramètres, questions)
Sécurisation : détection de patterns suspects, nettoyage
Support multi-providers (Mistral, OpenAI)

4. performance.py - Monitoring des performances

PerformanceMonitor : mesure automatique des performances
Métriques : temps, mémoire, CPU, taux de succès
Décorateur pour instrumenter les fonctions
Export et historique des métriques

## .env.example :

# .env.example
MISTRAL_API_KEY=your_mistral_api_key_here

# Installation et lancement
# 1. Créer un environnement virtuel
python -m venv venv
source venv/bin/activate  # Linux/Mac
# ou
venv\Scripts\activate  # Windows

# 2. Installer les dépendances
pip install gradio>=4.0.0 langchain>=0.1.0 langchain-community langchain-mistralai chromadb>=0.4.0 mistralai>=0.1.0 PyPDF2>=3.0.0 python-docx>=0.8.11 sentence-transformers>=2.2.0 python-dotenv>=1.0.0 tiktoken>=0.5.0

# 3. Créer le fichier .env avec votre clé API
cp .env.example .env
# Puis éditer .env avec votre vraie clé API Mistral

# 4. Lancer l'application
python app/main.py

# Structure des dossiers à créer
mkdir -p app/components
mkdir -p data/uploads
mkdir -p data/vectorstore
mkdir -p utils

# Mini-récap :

### ✨ Fonctionnalités principales :
- **Interface Gradio moderne** avec 3 onglets (Configuration, Chat, Aide)
- **Mistral AI intégré** (mistral-tiny pour commencer, gratuit)
- **Embeddings gratuits** avec sentence-transformers (multilingue FR/EN)
- **Support PDF, DOCX, TXT** avec extraction robuste
- **ChromaDB local** pour la persistance
- **Historique de conversation** avec sources
- **Configuration flexible** des paramètres

### 🚀 Pour lancer l'application :

1. **Créez la structure** :
```bash
mkdir document-chatbot && cd document-chatbot
mkdir -p app/components data/uploads data/vectorstore
```

2. **Copiez le code** dans les fichiers appropriés
3. **Installez les dépendances** avec pip
4. **Obtenez votre clé API** Mistral (gratuite) sur [console.mistral.ai](https://console.mistral.ai/)
5. **Lancez** : `python app/main.py`

### 🔧 Points forts de cette implémentation :

- **100% gratuit** (sauf limite API Mistral gratuite)
- **Multilingue** (français/anglais optimisé)
- **Robuste** avec gestion d'erreurs complète
- **Modulaire** et extensible
- **Interface intuitive** avec aide intégrée

L'application sera accessible sur `http://localhost:7860` et ressemblera à SecInsights.ai avec des fonctionnalités similaires !

Voulez-vous que je vous aide avec l'installation ou que j'ajoute des fonctionnalités spécifiques ?

# Ok NEW GAME :

# 🚀 Guide d'installation et utilisation complète

## 📦 Installation

### 1. Créer la structure du projet
```bash
mkdir document-chatbot
cd document-chatbot

# Créer l'arborescence
mkdir -p app/components
mkdir -p utils
mkdir -p data/uploads
mkdir -p data/vectorstore
mkdir -p data/exports
```

### 2. Installer les dépendances
```bash
# Créer un environnement virtuel
python -m venv venv

# Activer l'environnement
# Linux/Mac:
source venv/bin/activate
# Windows:
venv\Scripts\activate

# Installer les packages
pip install gradio>=4.0.0
pip install langchain>=0.1.0
pip install langchain-community
pip install langchain-mistralai
pip install chromadb>=0.4.0
pip install mistralai>=0.1.0
pip install PyPDF2>=3.0.0
pip install python-docx>=0.8.11
pip install sentence-transformers>=2.2.0
pip install python-dotenv>=1.0.0
pip install tiktoken>=0.5.0
pip install psutil>=5.9.0
pip install python-magic>=0.4.27

# Alternative pour python-magic sur Windows :
# pip install python-magic-bin==0.4.14
```

### 3. Configuration
```bash
# Créer le fichier d'environnement
cp .env.example .env

# Éditer .env avec votre clé API Mistral
echo "MISTRAL_API_KEY=votre_cle_api_ici" > .env
```

## 🎯 Utilisation

### Version Simple (originale)
```bash
python app/main.py
```

### Version Avancée (avec tous les utilitaires)
```bash
python app/main_enhanced.py
```

## 🆕 Nouvelles fonctionnalités de la version améliorée

### 1. **Découpage intelligent de documents**
- **Détection automatique** du type de document
- **Types supportés** : 
  - 📚 Académique (articles, thèses, rapports de recherche)
  - 🔧 Technique (manuels, documentation, guides)
  - ⚖️ Légal (contrats, règlements, lois)
  - 📄 Général (tous autres documents)

- **Optimisations par type** :
  - Séparateurs spécialisés
  - Préservation de la structure
  - Extraction de mots-clés
  - Métadonnées enrichies

### 2. **Templates de prompts optimisés**
- **Sélection automatique** du template selon le document
- **Templates disponibles** :
  - `general` : Usage standard
  - `academic` : Analyse rigoureuse, citations
  - `technical` : Réponses pratiques, étapes
  - `legal` : Analyse structurée, articles
  - `summary` : Résumés structurés
  - `extraction` : Extraction d'informations

### 3. **Validation et sécurité renforcées**
- **Validation des fichiers** :
  - Vérification du format et de la taille
  - Détection du type MIME
  - Protection contre les fichiers malveillants

- **Validation des entrées** :
  - Clés API (format et longueur)
  - Paramètres (plages valides)
  - Questions (nettoyage et sécurisation)

### 4. **Monitoring de performance**
- **Métriques en temps réel** :
  - Temps de traitement
  - Utilisation mémoire et CPU
  - Taux de succès des opérations

- **Historique et export** :
  - Sauvegarde des métriques
  - Export des sessions
  - Analyse des performances

## 🎛️ Interface utilisateur améliorée

### Onglet "Configuration Avancée"
1. **Clé API** avec validation automatique
2. **Upload de fichier** avec vérification en temps réel
3. **Paramètres avancés** :
   - Taille des chunks (optimisée automatiquement)
   - Nombre de documents récupérés
   - **Type de template** (auto-détection ou manuel)
4. **Informations détaillées** :
   - Type de document détecté
   - Nombre de chunks créés
   - Mots-clés extraits
   - Métriques de performance

### Onglet "Chat Intelligent"
1. **Conversation améliorée** avec :
   - Sources détaillées avec aperçu
   - Mots-clés des chunks utilisés
   - Template de prompt utilisé
2. **Métadonnées en temps réel** :
   - Nombre de sources consultées
   - Temps de réponse
   - Template utilisé
3. **Validation automatique** des questions

### Onglet "Monitoring"
1. **Statut du système** :
   - Configuration actuelle
   - Ressources système (CPU, mémoire, disque)
   - Templates disponibles
2. **Export de session** :
   - Historique des conversations
   - Métriques de performance
   - Configuration utilisée

## 🔍 Exemples d'utilisation

### Document académique
```
Document : Article de recherche sur l'IA
Type détecté : academic
Template utilisé : academic
Réponse : "Selon l'étude présentée dans ce document, les auteurs démontrent que..."
```

### Documentation technique
```
Document : Manuel d'installation logiciel
Type détecté : technical
Template utilisé : technical
Réponse : "Pour installer le logiciel, suivez ces étapes : 1. Télécharger..."
```

### Document légal
```
Document : Contrat de travail
Type détecté : legal
Template utilisé : legal
Réponse : "L'article 3.2 du contrat stipule que..."
```

## 📊 Métriques et performance

### Métriques collectées
- **Temps de traitement** par opération
- **Utilisation mémoire** (avant/après)
- **CPU** pendant le traitement
- **Taux de succès** des opérations
- **Erreurs** avec détails

### Export des données
```json
{
  "document": "mon_document.pdf",
  "system_status": {
    "template_used": "academic",
    "conversation_length": 5,
    "performance_summary": {...}
  },
  "conversation_history": [...],
  "performance_metrics": {...}
}
```

## 🎯 Avantages de la version améliorée

### 🚀 Performance
- **Découpage optimisé** selon le type de document
- **Templates spécialisés** pour de meilleures réponses
- **Monitoring en temps réel** des performances

### 🔒 Sécurité
- **Validation robuste** des entrées
- **Nettoyage automatique** des données
- **Protection** contre les injections

### 🧠 Intelligence
- **Détection automatique** du contexte
- **Adaptation dynamique** des réponses
- **Métadonnées enrichies** pour plus de précision

### 📈 Observabilité
- **Métriques détaillées** en temps réel
- **Historique complet** des performances
- **Export facile** des données

## 🔧 Personnalisation

### Ajouter un nouveau template
```python
from utils.prompt_templates import PromptTemplateManager

manager = PromptTemplateManager()
manager.create_custom_template(
    template_name="medical",
    template_content="Tu es un assistant médical...",
    input_variables=["context", "question"]
)
```

### Personnaliser la détection de type
```python
from utils.text_splitter import SmartTextSplitter

splitter = SmartTextSplitter()
# Ajouter des mots-clés pour un nouveau type
splitter.separators['medical'] = ["\n\n", "\n", ". ", "; ", " ", ""]
```

## 🐛 Dépannage

### Problèmes courants

1. **Erreur d'installation de python-magic** :
   ```bash
   # Linux
   sudo apt-get install libmagic1
   
   # macOS
   brew install libmagic
   
   # Windows
   pip install python-magic-bin
   ```

2. **Erreur de clé API Mistral** :
   - Vérifiez que la clé commence par le bon format
   - Consultez https://console.mistral.ai/ pour obtenir une clé

3. **Problème de mémoire** :
   - Réduisez la taille des chunks
   - Diminuez le nombre de documents récupérés
   - Surveillez l'onglet Monitoring

4. **Documents non traités** :
   - Vérifiez le format (PDF, DOCX, TXT)
   - Assurez-vous que le fichier contient du texte
   - Vérifiez la taille (max 10MB)

## 🎉 Fonctionnalités futures

- 🌐 **Support multilingue** avancé
- 🔗 **Intégration d'APIs** externes
- 📱 **Interface mobile** responsive
- 🤖 **Agent autonome** pour l'analyse
- 📊 **Tableaux de bord** avancés
- 💾 **Base de données** persistante
- 🔄 **Synchronisation** multi-utilisateurs

## 📁 Structure finale du projet :

In [ ]:
document-chatbot/
├── app/
│   ├── main.py                    # Version originale
│   ├── main_enhanced.py           # Version améliorée avec utilitaires
│   ├── config.py
│   └── components/
│       ├── file_processor.py
│       ├── vectorstore.py         # Version originale
│       ├── vectorstore_enhanced.py # Version améliorée
│       ├── chat_engine.py         # Version originale
│       ├── chat_engine_enhanced.py # Version améliorée
│       └── memory.py
├── utils/
│   ├── __init__.py               # ✅ Créé
│   ├── text_splitter.py          # ✅ Créé
│   ├── prompt_templates.py       # ✅ Créé
│   ├── validators.py             # ✅ Créé
│   └── performance.py            # ✅ Créé
├── data/
│   ├── uploads/
│   ├── vectorstore/
│   └── exports/                  # Nouveau : exports de session
├── requirements.txt
├── .env.example
└── README.md

## Enhanced scripts

In [ ]:
# app/components/vectorstore_enhanced.py
# Version améliorée avec les utilitaires

import os
from typing import List, Optional, Tuple
import chromadb
from chromadb.config import Settings
from langchain_community.embeddings import HuggingFaceEmbeddings
from langchain_community.vectorstores import Chroma
from langchain.schema import Document

from app.config import Config
from utils.text_splitter import SmartTextSplitter
from utils.validators import InputValidator
from utils.performance import PerformanceMonitor

class EnhancedVectorStoreManager:
    """Version améliorée du gestionnaire de base vectorielle avec utilitaires"""
    
    def __init__(self):
        self.embeddings = None
        self.vectorstore = None
        self.text_splitter = SmartTextSplitter()
        self.performance_monitor = PerformanceMonitor()
        self._setup_embeddings()
    
    def _setup_embeddings(self):
        """Configure les embeddings avec monitoring"""
        @self.performance_monitor.measure_performance("setup_embeddings")
        def _setup():
            model_name = "sentence-transformers/paraphrase-multilingual-MiniLM-L12-v2"
            self.embeddings = HuggingFaceEmbeddings(
                model_name=model_name,
                model_kwargs={'device': 'cpu'},
                encode_kwargs={'normalize_embeddings': True}
            )
        
        _setup()
    
    @property
    def performance_metrics(self):
        """Accès aux métriques de performance"""
        return self.performance_monitor.get_metrics_summary()
    
    def initialize_vectorstore(self, collection_name: str = Config.COLLECTION_NAME):
        """Initialise la base vectorielle avec monitoring"""
        @self.performance_monitor.measure_performance("initialize_vectorstore")
        def _initialize():
            os.makedirs(Config.CHROMADB_PATH, exist_ok=True)
            client = chromadb.PersistentClient(path=Config.CHROMADB_PATH)
            
            try:
                client.delete_collection(collection_name)
            except ValueError:
                pass
            
            self.vectorstore = Chroma(
                client=client,
                collection_name=collection_name,
                embedding_function=self.embeddings,
            )
        
        _initialize()
    
    def add_documents_enhanced(self, 
                             text: str, 
                             filename: str, 
                             chunk_size: int, 
                             chunk_overlap: int) -> Tuple[int, dict]:
        """
        Version améliorée d'ajout de documents avec validation et monitoring
        Returns: (nombre_chunks, informations_détaillées)
        """
        # Validation des paramètres
        valid_chunk, chunk_error = InputValidator.validate_chunk_size(chunk_size)
        if not valid_chunk:
            raise ValueError(f"Chunk size invalide: {chunk_error}")
        
        @self.performance_monitor.measure_performance("add_documents")
        def _add_documents():
            if not self.vectorstore:
                raise ValueError("VectorStore non initialisé")
            
            # Découpage intelligent avec métadonnées enrichies
            documents = self.text_splitter.split_documents_with_metadata(
                text, filename, chunk_size, chunk_overlap
            )
            
            # Ajout à la base vectorielle
            self.vectorstore.add_documents(documents)
            
            # Informations détaillées sur le traitement
            doc_info = {
                "total_chunks": len(documents),
                "document_type": documents[0].metadata.get("document_type", "unknown") if documents else "unknown",
                "average_chunk_size": sum(len(doc.page_content) for doc in documents) / len(documents) if documents else 0,
                "keywords_extracted": any("keywords" in doc.metadata for doc in documents),
                "structure_preserved": sum(1 for doc in documents if doc.metadata.get("contains_structure", False))
            }
            
            return len(documents), doc_info
        
        return _add_documents()
    
    def search_with_metadata(self, query: str, k: int = Config.DEFAULT_K_DOCUMENTS) -> List[dict]:
        """Recherche avec métadonnées détaillées"""
        # Validation de la requête
        valid_query, query_error = InputValidator.validate_question(query)
        if not valid_query:
            raise ValueError(f"Requête invalide: {query_error}")
        
        @self.performance_monitor.measure_performance("search_documents")
        def _search():
            if not self.vectorstore:
                return []
            
            documents = self.vectorstore.similarity_search(query, k=k)
            
            # Enrichissement des résultats avec métadonnées
            enriched_results = []
            for doc in documents:
                result = {
                    "content": doc.page_content,
                    "metadata": doc.metadata,
                    "relevance_score": getattr(doc, 'relevance_score', None),
                    "chunk_info": {
                        "position": doc.metadata.get("chunk_position", "unknown"),
                        "total_chunks": doc.metadata.get("total_chunks", 0),
                        "chunk_id": doc.metadata.get("chunk_id", 0),
                        "has_structure": doc.metadata.get("contains_structure", False),
                        "keywords": doc.metadata.get("keywords", [])
                    }
                }
                enriched_results.append(result)
            
            return enriched_results
        
        return _search()

# app/components/chat_engine_enhanced.py
# Version améliorée du moteur de chat

from typing import Optional, Tuple, List, Dict
from langchain_mistralai import ChatMistralAI
from langchain.chains import ConversationalRetrievalChain
from langchain.memory import ConversationBufferMemory

from app.components.vectorstore_enhanced import EnhancedVectorStoreManager
from app.components.memory import ConversationMemory
from utils.prompt_templates import PromptTemplateManager
from utils.validators import InputValidator
from utils.performance import PerformanceMonitor

class EnhancedChatEngine:
    """Moteur de conversation amélioré avec utilitaires avancés"""
    
    def __init__(self):
        self.llm = None
        self.vectorstore_manager = EnhancedVectorStoreManager()
        self.memory = ConversationMemory()
        self.prompt_manager = PromptTemplateManager()
        self.performance_monitor = PerformanceMonitor()
        self.chain = None
        self.current_template_type = 'general'
    
    def setup_llm(self, api_key: str, model: str = "mistral-tiny"):
        """Configuration du LLM avec validation"""
        # Validation de la clé API
        valid_key, key_error = InputValidator.validate_api_key(api_key, 'mistral')
        if not valid_key:
            raise ValueError(f"Clé API invalide: {key_error}")
        
        @self.performance_monitor.measure_performance("setup_llm")
        def _setup():
            self.llm = ChatMistralAI(
                mistral_api_key=api_key,
                model=model,
                temperature=0.1,
                max_tokens=1000
            )
        
        _setup()
    
    def setup_chain_with_template(self, 
                                 k_documents: int = 3, 
                                 template_type: str = 'auto'):
        """Configuration de la chaîne avec template intelligent"""
        if not self.llm:
            raise ValueError("LLM non configuré")
        
        if not self.vectorstore_manager.vectorstore:
            raise ValueError("VectorStore non initialisé")
        
        # Validation des paramètres
        valid_k, k_error = InputValidator.validate_k_documents(k_documents)
        if not valid_k:
            raise ValueError(f"Paramètre k invalide: {k_error}")
        
        @self.performance_monitor.measure_performance("setup_chain")
        def _setup():
            # Détection automatique du type de template si nécessaire
            if template_type == 'auto':
                # Récupérer le type de document depuis les métadonnées
                sample_docs = self.vectorstore_manager.search_with_metadata("test", k=1)
                if sample_docs:
                    document_type = sample_docs[0]['metadata'].get('document_type', 'general')
                    template_type_final = document_type
                else:
                    template_type_final = 'general'
            else:
                template_type_final = template_type
            
            # Sélection du template optimisé
            prompt_template = self.prompt_manager.get_optimized_template(
                document_type=template_type_final,
                query_type='general'
            )
            
            self.current_template_type = template_type_final
            
            # Configuration de la mémoire
            memory = ConversationBufferMemory(
                memory_key="chat_history",
                return_messages=True,
                output_key="answer"
            )
            
            # Création de la chaîne
            self.chain = ConversationalRetrievalChain.from_llm(
                llm=self.llm,
                retriever=self.vectorstore_manager.get_retriever(k=k_documents),
                memory=memory,
                return_source_documents=True,
                combine_docs_chain_kwargs={"prompt": prompt_template}
            )
        
        _setup()
    
    def process_question_enhanced(self, question: str) -> Tuple[str, List[str], Dict]:
        """
        Traitement avancé des questions avec métadonnées enrichies
        Returns: (answer, sources, metadata)
        """
        # Validation et nettoyage de la question
        valid_question, question_error = InputValidator.validate_question(question)
        if not valid_question:
            return f"Question invalide: {question_error}", [], {}
        
        question = InputValidator.sanitize_input(question)
        
        @self.performance_monitor.measure_performance("process_question")
        def _process():
            if not self.chain:
                return "Erreur: Système non configuré", [], {}
            
            try:
                # Exécution de la chaîne
                result = self.chain({"question": question})
                
                answer = result.get("answer", "Pas de réponse générée")
                
                # Extraction enrichie des sources
                sources = []
                source_metadata = []
                
                if "source_documents" in result:
                    for doc in result["source_documents"]:
                        filename = doc.metadata.get("filename", "Document")
                        chunk_id = doc.metadata.get("chunk_id", 0)
                        chunk_position = doc.metadata.get("chunk_position", "unknown")
                        keywords = doc.metadata.get("keywords", [])
                        
                        source_label = f"{filename} (section {chunk_id + 1})"
                        sources.append(source_label)
                        
                        source_metadata.append({
                            "filename": filename,
                            "chunk_id": chunk_id,
                            "position": chunk_position,
                            "keywords": keywords,
                            "content_preview": doc.page_content[:100] + "..." if len(doc.page_content) > 100 else doc.page_content
                        })
                
                # Métadonnées de la réponse
                response_metadata = {
                    "template_used": self.current_template_type,
                    "sources_count": len(sources),
                    "source_details": source_metadata,
                    "performance": self.performance_monitor.get_metrics_summary("process_question", last_n_minutes=1)
                }
                
                # Sauvegarde dans l'historique
                self.memory.add_exchange(question, answer, sources)
                
                return answer, sources, response_metadata
                
            except Exception as e:
                error_msg = f"Erreur lors du traitement: {str(e)}"
                return error_msg, [], {"error": str(e)}
        
        return _process()
    
    def get_system_status(self) -> Dict:
        """Retourne le statut détaillé du système"""
        return {
            "llm_configured": self.llm is not None,
            "vectorstore_ready": self.vectorstore_manager.vectorstore is not None,
            "chain_ready": self.chain is not None,
            "current_template": self.current_template_type,
            "conversation_length": len(self.memory.conversation_history),
            "performance_summary": self.performance_monitor.get_metrics_summary(),
            "system_info": self.performance_monitor.get_system_info(),
            "available_templates": self.prompt_manager.get_available_templates()
        }
    
    def switch_template(self, new_template_type: str) -> bool:
        """Change le template de prompt en cours d'exécution"""
        try:
            available_templates = self.prompt_manager.get_available_templates()
            if new_template_type not in available_templates:
                return False
            
            # Reconfigurer la chaîne avec le nouveau template
            if self.chain:
                k_current = getattr(self.chain.retriever, 'search_kwargs', {}).get('k', 3)
                self.setup_chain_with_template(k_current, new_template_type)
                return True
            
            return False
        except Exception:
            return False

# app/main_enhanced.py
# Version améliorée de l'application principale

import gradio as gr
import os
from typing import Optional, Tuple, List
import json

from app.components.file_processor import FileProcessor
from app.components.chat_engine_enhanced import EnhancedChatEngine
from app.config import Config
from utils.validators import FileValidator, InputValidator
from utils.performance import PerformanceMonitor

class EnhancedDocumentChatBot:
    """Application principale améliorée avec tous les utilitaires"""
    
    def __init__(self):
        self.chat_engine = EnhancedChatEngine()
        self.global_monitor = PerformanceMonitor()
        self.current_document = None
        self.document_processed = False
        self.system_status = {}
    
    def process_document_enhanced(self, 
                                api_key: str, 
                                file_obj, 
                                chunk_size: int, 
                                k_documents: int,
                                template_type: str = 'auto') -> Tuple[str, str, str]:
        """
        Version améliorée du traitement de documents
        Returns: (status_message, document_info, performance_info)
        """
        @self.global_monitor.measure_performance("full_document_processing")
        def _process():
            try:
                # Validation du fichier
                file_valid, file_error = FileValidator.validate_file(file_obj)
                if not file_valid:
                    return f"❌ {file_error}", "", ""
                
                # Validation de la clé API
                api_valid, api_error = InputValidator.validate_api_key(api_key.strip(), 'mistral')
                if not api_valid:
                    return f"❌ {api_error}", "", ""
                
                # Configuration du LLM
                self.chat_engine.setup_llm(api_key.strip())
                
                # Traitement du fichier
                text_content, filename = FileProcessor.process_uploaded_file(file_obj)
                
                # Initialisation de la base vectorielle
                self.chat_engine.vectorstore_manager.initialize_vectorstore()
                
                # Ajout des documents avec métadonnées enrichies
                chunk_overlap = max(50, chunk_size // 5)
                num_chunks, doc_details = self.chat_engine.vectorstore_manager.add_documents_enhanced(
                    text_content, filename, chunk_size, chunk_overlap
                )
                
                # Configuration de la chaîne avec template intelligent
                self.chat_engine.setup_chain_with_template(k_documents, template_type)
                
                # Mise à jour de l'état
                self.current_document = filename
                self.document_processed = True
                self.system_status = self.chat_engine.get_system_status()
                
                # Messages de statut
                status = f"✅ Document traité avec succès!"
                
                doc_info = f"""
### 📄 Informations du document
- **Fichier**: {filename}
- **Type détecté**: {doc_details['document_type']}
- **Chunks créés**: {num_chunks}
- **Taille moyenne des chunks**: {doc_details['average_chunk_size']:.0f} caractères
- **Structure préservée**: {doc_details['structure_preserved']} sections
- **Mots-clés extraits**: {'✅' if doc_details['keywords_extracted'] else '❌'}

### ⚙️ Configuration
- **Template utilisé**: {self.chat_engine.current_template_type}
- **Documents récupérés**: {k_documents}
- **Modèle**: Mistral AI (mistral-tiny)
                """.strip()
                
                # Informations de performance
                perf_summary = self.global_monitor.get_metrics_summary()
                system_info = self.global_monitor.get_system_info()
                
                perf_info = f"""
### 📊 Performance
- **Temps de traitement**: {perf_summary.get('avg_duration', 0):.2f}s
- **Utilisation mémoire**: {system_info['memory_percent']:.1f}%
- **CPU**: {system_info['cpu_percent']:.1f}%
- **Taux de succès**: {perf_summary.get('success_rate', 100):.1f}%
                """.strip()
                
                return status, doc_info, perf_info
                
            except Exception as e:
                error_msg = f"❌ Erreur: {str(e)}"
                return error_msg, "", ""
        
        return _process()
    
    def chat_enhanced(self, 
                     message: str, 
                     history: List[List[str]]) -> Tuple[str, List[List[str]], str]:
        """
        Version améliorée du chat avec métadonnées
        Returns: ("", updated_history, metadata_info)
        """
        if not self.document_processed:
            history.append([message, "⚠️ Veuillez d'abord traiter un document dans l'onglet Configuration."])
            return "", history, ""
        
        if not message.strip():
            return "", history, ""
        
        # Traitement amélioré de la question
        answer, sources, metadata = self.chat_engine.process_question_enhanced(message)
        
        # Formatage de la réponse avec sources détaillées
        if sources:
            source_details = []
            for i, source in enumerate(sources):
                if i < len(metadata.get('source_details', [])):
                    detail = metadata['source_details'][i]
                    keywords_str = ', '.join(detail['keywords'][:3]) if detail['keywords'] else 'N/A'
                    source_details.append(f"📄 {source} | 🏷️ Mots-clés: {keywords_str}")
                else:
                    source_details.append(f"📄 {source}")
            
            formatted_answer = f"{answer}\n\n**Sources consultées:**\n" + "\n".join(source_details)
        else:
            formatted_answer = answer
        
        # Informations de métadonnées pour affichage
        metadata_info = f"""
**Template**: {metadata.get('template_used', 'N/A')} | **Sources**: {metadata.get('sources_count', 0)} | **Performance**: {metadata.get('performance', {}).get('avg_duration', 0):.2f}s
        """.strip()
        
        # Mise à jour de l'historique
        history.append([message, formatted_answer])
        
        return "", history, metadata_info
    
    def get_detailed_status(self) -> str:
        """Retourne un statut détaillé du système"""
        if not self.document_processed:
            return "🔴 **Statut**: Aucun document traité"
        
        status = self.chat_engine.get_system_status()
        
        status_text = f"""
### 🟢 Système opérationnel

**Configuration actuelle:**
- Template: {status['current_template']}
- Conversations: {status['conversation_length']} échanges
- Performance moyenne: {status['performance_summary'].get('avg_duration', 0):.2f}s

**Ressources système:**
- Mémoire: {status['system_info']['memory_percent']:.1f}%
- CPU: {status['system_info']['cpu_percent']:.1f}%
- Espace disque: {status['system_info']['disk_usage_percent']:.1f}%

**Templates disponibles:** {', '.join(status['available_templates'])}
        """.strip()
        
        return status_text
    
    def export_session_data(self) -> str:
        """Exporte les données de la session"""
        if not self.document_processed:
            return "Aucune donnée à exporter"
        
        try:
            # Données à exporter
            session_data = {
                "document": self.current_document,
                "system_status": self.chat_engine.get_system_status(),
                "conversation_history": self.chat_engine.memory.conversation_history,
                "performance_metrics": self.global_monitor.get_metrics_summary()
            }
            
            # Sauvegarde dans un fichier
            filename = f"session_export_{self.current_document}_{datetime.now().strftime('%Y%m%d_%H%M%S')}.json"
            
            with open(filename, 'w', encoding='utf-8') as f:
                json.dump(session_data, f, indent=2, ensure_ascii=False, default=str)
            
            return f"✅ Session exportée: {filename}"
            
        except Exception as e:
            return f"❌ Erreur d'export: {str(e)}"

# Integration dans l'interface Gradio
def create_enhanced_interface():
    """Crée l'interface Gradio améliorée"""
    
    app = EnhancedDocumentChatBot()
    
    # CSS amélioré
    css = """
    .gradio-container {
        font-family: 'Segoe UI', Tahoma, Geneva, Verdana, sans-serif;
        max-width: 1200px;
        margin: 0 auto;
    }
    .tab-nav button {
        font-size: 16px;
        font-weight: 500;
    }
    .performance-info {
        background: #f0f0f0;
        padding: 10px;
        border-radius: 5px;
        font-size: 12px;
        color: #666;
    }
    """
    
    with gr.Blocks(css=css, title="ChatBot Documentaire Avancé - Mistral AI", theme=gr.themes.Soft()) as interface:
        
        gr.Markdown("""
        # 📄 ChatBot Documentaire Avancé avec Mistral AI
        
        **Nouvelle version** avec découpage intelligent, templates optimisés et monitoring de performance !
        """)
        
        with gr.Tabs():
            # Onglet Configuration Avancé
            with gr.Tab("⚙️ Configuration Avancée", id="config"):
                
                with gr.Row():
                    with gr.Column(scale=2):
                        api_key_input = gr.Textbox(
                            label="🔑 Clé API Mistral AI",
                            placeholder="Saisissez votre clé API Mistral...",
                            type="password",
                            info="Validation automatique de la clé"
                        )
                        
                        file_upload = gr.File(
                            label="📁 Fichier à analyser",
                            file_types=[".pdf", ".docx", ".txt"],
                            type="binary"
                        )
                        
                        with gr.Row():
                            chunk_size_slider = gr.Slider(
                                minimum=200,
                                maximum=2000,
                                value=Config.DEFAULT_CHUNK_SIZE,
                                step=100,
                                label="📏 Taille des chunks",
                                info="Découpage intelligent automatique"
                            )
                            
                            k_documents_slider = gr.Slider(
                                minimum=1,
                                maximum=10,
                                value=Config.DEFAULT_K_DOCUMENTS,
                                step=1,
                                label="🔍 Documents récupérés"
                            )
                        
                        template_dropdown = gr.Dropdown(
                            choices=['auto', 'general', 'academic', 'technical', 'legal'],
                            value='auto',
                            label="🎯 Type de template",
                            info="Auto = détection automatique"
                        )
                        
                        process_btn = gr.Button("🚀 Traiter le document", variant="primary", size="lg")
                    
                    with gr.Column(scale=1):
                        status_output = gr.Textbox(
                            label="📊 Statut",
                            interactive=False,
                            lines=2
                        )
                        
                        doc_info_output = gr.Markdown(
                            label="ℹ️ Informations détaillées"
                        )
                        
                        perf_info_output = gr.Markdown(
                            label="⚡ Performance",
                            elem_classes=["performance-info"]
                        )
            
            # Onglet Chat Amélioré
            with gr.Tab("💬 Chat Intelligent", id="chat"):
                
                chatbot = gr.Chatbot(
                    label="Conversation avec intelligence augmentée",
                    height=500,
                    show_copy_button=True
                )
                
                with gr.Row():
                    msg_input = gr.Textbox(
                        label="Votre question",
                        placeholder="Question intelligente avec validation automatique...",
                        scale=4
                    )
                    
                    with gr.Column(scale=1):
                        send_btn = gr.Button("📤 Envoyer", variant="primary")
                        clear_btn = gr.Button("🗑️ Effacer", variant="secondary")
                
                # Informations de métadonnées en temps réel
                metadata_display = gr.Textbox(
                    label="🔍 Métadonnées de la réponse",
                    interactive=False,
                    lines=1,
                    elem_classes=["performance-info"]
                )
            
            # Onglet Monitoring
            with gr.Tab("📊 Monitoring", id="monitoring"):
                
                with gr.Row():
                    refresh_btn = gr.Button("🔄 Actualiser", variant="secondary")
                    export_btn = gr.Button("💾 Exporter Session", variant="primary")
                
                system_status_display = gr.Markdown(
                    label="Statut du système",
                    value="Aucune donnée disponible"
                )
                
                export_result_display = gr.Textbox(
                    label="Résultat d'export",
                    interactive=False
                )
        
        # Événements
        process_btn.click(
            fn=app.process_document_enhanced,
            inputs=[api_key_input, file_upload, chunk_size_slider, k_documents_slider, template_dropdown],
            outputs=[status_output, doc_info_output, perf_info_output]
        )
        
        send_btn.click(
            fn=app.chat_enhanced,
            inputs=[msg_input, chatbot],
            outputs=[msg_input, chatbot, metadata_display]
        )
        
        msg_input.submit(
            fn=app.chat_enhanced,
            inputs=[msg_input, chatbot],
            outputs=[msg_input, chatbot, metadata_display]
        )
        
        clear_btn.click(
            fn=lambda: ([], ""),
            outputs=[chatbot, metadata_display]
        )
        
        refresh_btn.click(
            fn=app.get_detailed_status,
            outputs=[system_status_display]
        )
        
        export_btn.click(
            fn=app.export_session_data,
            outputs=[export_result_display]
        )
    
    return interface

if __name__ == "__main__":
    from datetime import datetime
    os.makedirs("./data/uploads", exist_ok=True)
    os.makedirs("./data/vectorstore", exist_ok=True)
    
    interface = create_enhanced_interface()
    interface.launch(
        server_name="0.0.0.0",
        server_port=7860,
        share=False,
        debug=True
    )

🚀 Intégration complète des utilitaires !
Voici comment les utilitaires sont intégrés dans votre application :
✨ Nouvelles fonctionnalités ajoutées :
1. Découpage intelligent (SmartTextSplitter)

Détection automatique du type de document (académique, technique, légal)
Préservation de structure avec mots-clés extraits
Métadonnées enrichies pour chaque chunk

2. Templates de prompts optimisés (PromptTemplateManager)

6 templates spécialisés selon le type de contenu
Sélection automatique ou manuelle
Optimisation contextuelle des réponses

3. Validation robuste (Validators)

Validation des fichiers : taille, format, sécurité
Validation des entrées : clés API, paramètres, questions
Nettoyage automatique des inputs utilisateur

4. Monitoring de performance (PerformanceMonitor)

Métriques en temps réel : temps, mémoire, CPU
Historique des performances avec export
Surveillance système automatique

🆕 Interface améliorée :
Nouveaux onglets :

Configuration Avancée : sélection de templates, validation en temps réel
Chat Intelligent : métadonnées des réponses, validation des questions
Monitoring : statut système, export de session

Nouvelles informations affichées :

Type de document détecté automatiquement
Template de prompt utilisé
Mots-clés extraits des chunks
Métriques de performance en temps réel
Sources détaillées avec prévisualisation

```
document-chatbot/
├── app/
│   ├── main.py                    # Version originale
│   ├── main_enhanced.py           # Version améliorée avec utilitaires
│   ├── config.py
│   └── components/
│       ├── file_processor.py
│       ├── vectorstore.py         # Version originale
│       ├── vectorstore_enhanced.py # Version améliorée
│       ├── chat_engine.py         # Version originale
│       ├── chat_engine_enhanced.py # Version améliorée
│       └── memory.py
├── utils/
│   ├── __init__.py               # ✅ Créé
│   ├── text_splitter.py          # ✅ Créé
│   ├── prompt_templates.py       # ✅ Créé
│   ├── validators.py             # ✅ Créé
│   └── performance.py            # ✅ Créé
├── data/
│   ├── uploads/
│   ├── vectorstore/
│   └── exports/                  # Nouveau : exports de session
├── requirements.txt
├── .env.example
└── README.md
```

## 🔧 **Installation et utilisation :**## 🎯 **Résumé de votre projet complet**

Vous avez maintenant **deux versions** de votre ChatBot documentaire :

### 📱 **Version Simple** (`main.py`)
- Interface Gradio basique
- Fonctionnalités essentielles
- Facile à déployer et utiliser

### 🚀 **Version Avancée** (`main_enhanced.py`)
- **Découpage intelligent** avec détection automatique du type
- **Templates optimisés** pour chaque contexte
- **Validation robuste** et sécurité renforcée
- **Monitoring complet** des performances
- **Interface enrichie** avec 3 onglets spécialisés

## 🔥 **Points forts de la version avancée :**

1. **🧠 Intelligence augmentée** : détection automatique + templates spécialisés
2. **⚡ Performance optimisée** : monitoring en temps réel + métriques détaillées  
3. **🔒 Sécurité renforcée** : validation complète + nettoyage automatique
4. **📊 Observabilité totale** : export de sessions + historique complet
5. **🎯 Réponses de qualité** : adaptation contextuelle + sources enrichies

## 🚀 **Pour commencer :**

1. **Choisissez votre version** (simple ou avancée)
2. **Installez les dépendances** avec pip
3. **Obtenez votre clé Mistral AI** (gratuite)
4. **Lancez l'application** : `python app/main_enhanced.py`
5. **Accédez à** : `http://localhost:7860`

Votre ChatBot documentaire est maintenant **prêt à rivaliser avec SecInsights.ai** ! 🎉

Avez-vous des questions sur l'installation ou voulez-vous que j'ajoute d'autres fonctionnalités spécifiques ?